In [1]:
import os
import pprint
import pandas as pd
import tensorflow as tf
import tensorflow_model_analysis as tfma
from tfx import v1 as tfx
from absl import logging as absl_logging
from tfx.proto import example_gen_pb2, trainer_pb2
from apache_beam import logging as beam_logging
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
tf.get_logger().propagate = False
tf.get_logger().setLevel('ERROR')
absl_logging.set_verbosity('ERROR')
beam_logging.getLogger().setLevel('ERROR')
pp = pprint.PrettyPrinter()

2024-06-01 21:18:48.085774: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-01 21:18:48.217379: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-01 21:18:48.217434: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-01 21:18:48.219647: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-01 21:18:48.249926: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-01 21:18:48.250749: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
_pipeline_root = './pipeline/'
_data_root = './data/'

context = InteractiveContext(pipeline_root=_pipeline_root)

In [3]:
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=6),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2),
        example_gen_pb2.SplitConfig.Split(name='test', hash_buckets=2)
    ]))

In [4]:
example_gen = tfx.components.CsvExampleGen(input_base=_data_root, output_config=output)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [5]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval", "test"] ./pipeline/CsvExampleGen/examples/1


In [6]:
statistics_gen = tfx.components.StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [7]:
context.show(statistics_gen.outputs['statistics'])

In [8]:
schema_gen = tfx.components.SchemaGen(statistics=statistics_gen.outputs['statistics'], infer_feature_shape=False)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [9]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Age',INT,required,single,-
'Blast Furnace Slag',FLOAT,required,single,-
'Cement',FLOAT,required,single,-
'Coarse Aggregate',FLOAT,required,single,-
'Fine Aggregate',FLOAT,required,single,-
'Fly Ash',FLOAT,required,single,-
'Strength',FLOAT,required,single,-
'Superplasticizer',FLOAT,required,single,-
'Water',FLOAT,required,single,-


In [10]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [11]:
context.show(example_validator.outputs['anomalies'])

In [12]:
df = pd.read_csv('./data/concrete_data.csv')
df.dtypes

Cement                float64
Blast Furnace Slag    float64
Fly Ash               float64
Water                 float64
Superplasticizer      float64
Coarse Aggregate      float64
Fine Aggregate        float64
Age                     int64
Strength              float64
dtype: object

In [13]:
constants_module = 'constants.py'

In [14]:
%%writefile {constants_module}

LABEL_KEY = 'Strength'
NUMERICAL_FEATURES = ['Cement','Water', 'Blast Furnace Slag','Fly Ash','Water', 'Superplasticizer','Coarse Aggregate','Fine Aggregate','Age']


def t_name(key):
    return key + '_xf'

Writing constants.py


In [15]:
transform_module = 'transform.py'

In [16]:
%%writefile {transform_module}
import tensorflow as tf
import tensorflow_transform as tft

import constants

_NUMERICAL_FEATURES = constants.NUMERICAL_FEATURES
_LABEL_KEY = constants.LABEL_KEY


def fill_in_missing(x):
    default_value = '' if x.dtype == tf.string or False else 0
    if type(x) == tf.SparseTensor:
        x = tf.sparse.to_dense(
            tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
        default_value)
    return tf.squeeze(x, axis=1)
    
def preprocessing_fn(inputs):
    outputs={}
    for key in _NUMERICAL_FEATURES:
        scaled = tft.scale_to_0_1(fill_in_missing(inputs[key]))
        outputs[constants.t_name(key)] = tf.reshape(scaled, [-1])

    outputs[constants.t_name(_LABEL_KEY)] = fill_in_missing(inputs[_LABEL_KEY])
    return outputs

Writing transform.py


In [17]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(transform_module))
context.run(transform, enable_cache=False)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying transform.py -> build/lib
copying constants.py -> build/lib


/home/kali/miniconda3/envs/bie/lib/python3.9/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


installing to /tmp/tmpk6cah4tf
running install
running install_lib
copying build/lib/transform.py -> /tmp/tmpk6cah4tf
copying build/lib/constants.py -> /tmp/tmpk6cah4tf
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpk6cah4tf/tfx_user_code_Transform-0.0+d8aa27aa8f7255c8470ee54424889a8187225549f3dec31319c32c9b9506e4ae-py3.9.egg-info
running install_scripts
creating /tmp/tmpk6cah4tf/tfx_user_code_Transform-0.0+d8aa27aa8f7255c8470ee54424889a8187225549f3dec31319c32c9b9506e4ae.dist-info/WHEEL
creati

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [18]:
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

for tfrecord in dataset.take(1):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "Age_xf"
    value {
      float_list {
        value: 0.7390109896659851
      }
    }
  }
  feature {
    key: "Blast Furnace Slag_xf"
    value {
      float_list {
        value: 0.39649415016174316
      }
    }
  }
  feature {
    key: "Cement_xf"
    value {
      float_list {
        value: 0.52625572681427
      }
    }
  }
  feature {
    key: "Coarse Aggregate_xf"
    value {
      float_list {
        value: 0.3930392563343048
      }
    }
  }
  feature {
    key: "Fine Aggregate_xf"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "Fly Ash_xf"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "Strength_xf"
    value {
      float_list {
        value: 40.27000045776367
      }
    }
  }
  feature {
    key: "Superplasticizer_xf"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "Water_xf"
    value {
      float_list {
     

In [19]:
graph_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(graph_uri)

['metadata', 'transformed_metadata', 'transform_fn']

In [20]:
trainer_module = 'trainer.py'

In [21]:
%%writefile {trainer_module}
from tensorflow import keras
from typing import NamedTuple, Dict, Text, Any, List
from tfx.components.trainer.fn_args_utils import FnArgs, DataAccessor
import tensorflow as tf
import tensorflow_transform as tft
import constants
import os

device = "gpu"

if device == "tpu":
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
  tf.config.experimental_connect_to_cluster(resolver)
  tf.tpu.experimental.initialize_tpu_system(resolver)
  strategy = tf.distribute.experimental.TPUStrategy(resolver)
else:
  strategy = tf.distribute.MultiWorkerMirroredStrategy()

_NUMERICAL_FEATURES = constants.NUMERICAL_FEATURES
_LABEL_KEY = constants.LABEL_KEY
transformed_label_key = constants.t_name(_LABEL_KEY)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

def _gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def _input_fn(file_pattern,
              tf_transform_output,
              num_epochs=None,
              batch_size=32) -> tf.data.Dataset:

    transformed_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_label_key
        )

    return dataset

def _get_tf_examples_serving_signature(model, tf_transform_output):
  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def serve_tf_examples_fn(serialized_tf_examples):
      raw_feature_spec = tf_transform_output.raw_feature_spec()
      raw_feature_spec.pop(_LABEL_KEY)
      raw_features = tf.io.parse_example(serialized_tf_examples, raw_feature_spec)
      transformed_features = tf_transform_output.transform_features_layer()(raw_features)
      outputs = model(transformed_features)
      return {'outputs': outputs}

  return serve_tf_examples_fn

def get_model():
    for key in _NUMERICAL_FEATURES:
        input_numeric = []
        input_numeric.append(tf.keras.Input(shape=(1,),
            name=constants.t_name(key),
            dtype=tf.float32
        ))

        inputs = input_numeric
        wide = tf.keras.layers.concatenate(inputs)

        x = tf.keras.layers.Flatten()(wide)
        x = tf.keras.layers.Dense(128, activation='relu')(x)

        output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

        model = tf.keras.models.Model(inputs, output)

        model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics = ['mean_squared_error'])

        model.summary()

    return model

def run_fn(fn_args: FnArgs):
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
  train_dataset = _input_fn(fn_args.train_files[0], tf_transform_output, 10)
  eval_dataset = _input_fn(fn_args.eval_files[0], tf_transform_output, 10)

  model = get_model()

  log_dir = os.path.join(fn_args.model_run_dir, 'logs')
  os.makedirs(log_dir, exist_ok=True)
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, update_freq='batch')

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      callbacks=[tensorboard_callback])

  model.tft_layer = tf_transform_output.transform_features_layer()

  signatures = {
      'serving_default':
          _get_tf_examples_serving_signature(model, tf_transform_output)
  }

  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Writing trainer.py


In [22]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(trainer_module),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=50),
    eval_args=tfx.proto.EvalArgs(num_steps=50))

In [23]:
context.run(trainer, enable_cache=False)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying transform.py -> build/lib
copying trainer.py -> build/lib
copying constants.py -> build/lib
installing to /tmp/tmpn7q0ko1t
running install
running install_lib


/home/kali/miniconda3/envs/bie/lib/python3.9/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


copying build/lib/transform.py -> /tmp/tmpn7q0ko1t
copying build/lib/trainer.py -> /tmp/tmpn7q0ko1t
copying build/lib/constants.py -> /tmp/tmpn7q0ko1t
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpn7q0ko1t/tfx_user_code_Trainer-0.0+06867b8317b6dcf37dabfe65babc104e2795971b5ca111d0cef24b5dadf85a97-py3.9.egg-info
running install_scripts
creating /tmp/tmpn7q0ko1t/tfx_user_code_Trainer-0.0+06867b8317b6dcf37dabfe65babc104e2795971b5ca111d0cef24b5dadf85a97.dist-info/WHEEL
creating '/tmp/tmpiff34ddj/tfx_user_code_Tra

/home/kali/miniconda3/envs/bie/lib/python3.9/site-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['Blast Furnace Slag_xf', 'Cement_xf', 'Coarse Aggregate_xf', 'Fine Aggregate_xf', 'Fly Ash_xf', 'Superplasticizer_xf', 'Water_xf'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


50/50 [==============================] - 2s 19ms/step - loss: -208.7553 - mean_squared_error: 1544.6055 - val_loss: -599.8185 - val_mean_squared_error: 1363.7655


/home/kali/miniconda3/envs/bie/lib/python3.9/site-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['Fly Ash_xf', 'Coarse Aggregate_xf', 'Blast Furnace Slag_xf', 'Superplasticizer_xf', 'Cement_xf', 'Fine Aggregate_xf', 'Water_xf'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [24]:
model_artifact_dir = trainer.outputs['model'].get()[0].uri

pp.pprint(os.listdir(model_artifact_dir))

model_dir = os.path.join(model_artifact_dir, 'Format-Serving')

pp.pprint(os.listdir(model_dir))

['Format-Serving']
['fingerprint.pb', 'saved_model.pb', 'assets', 'keras_metadata.pb', 'variables']


In [25]:
model_run_artifact_dir = trainer.outputs['model_run'].get()[0].uri

pp.pprint(os.listdir(model_run_artifact_dir))

['logs']


In [26]:
tensorboard_logs_dir = os.path.join(model_run_artifact_dir, 'logs')
if os.path.exists(tensorboard_logs_dir):
    print("TensorBoard logs directory exists:", tensorboard_logs_dir)
    pp.pprint(os.listdir(tensorboard_logs_dir))
else:
    print("TensorBoard logs directory does not exist.")

TensorBoard logs directory exists: ./pipeline/Trainer/model_run/6/logs
['train', 'validation']


In [27]:
%load_ext tensorboard
%tensorboard --logdir {tensorboard_logs_dir}


In [28]:
import tensorflow_model_analysis as tfma
import constants

eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(
            signature_name='serving_default',
            label_key=constants.LABEL_KEY
        )
    ],
    metrics_specs=[tfma.MetricsSpec(
        metrics=[
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(
                class_name='MeanSquaredError',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(lower_bound={'value': 0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value': -1e-10}
                    )
                )
            )
        ]
    )],
    slicing_specs=[
        tfma.SlicingSpec(),
        tfma.SlicingSpec(
            feature_keys=['Age']
        )
    ]
)


In [29]:
model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')


context.run(model_resolver, enable_cache=True)


ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [30]:
evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
context.run(evaluator, enable_cache=True)

ExecutionResult(
    component_id: Evaluator
    execution_id: 8
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [31]:
model_resolver.outputs['model']

.type_name,Model
._artifacts,[]


In [32]:
evaluator.outputs.keys()

dict_keys(['evaluation', 'blessing'])

In [33]:
blessing_uri = evaluator.outputs['blessing'].get()[0].uri

os.listdir(blessing_uri)

['BLESSED']

In [34]:
context.run(model_resolver)

ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 9
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [35]:
model_resolver.outputs['model'].get()[0].uri

'./pipeline/Trainer/model/6'

In [36]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(trainer_module),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=500),
    eval_args=tfx.proto.EvalArgs(num_steps=200))

context.run(trainer, enable_cache=False)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying transform.py -> build/lib
copying trainer.py -> build/lib
copying constants.py -> build/lib
installing to /tmp/tmpi_7fapfg
running install
running install_lib
copying build/lib/transform.py -> /tmp/tmpi_7fapfg
copying build/lib/trainer.py -> /tmp/tmpi_7fapfg
copying build/lib/constants.py -> /tmp/tmpi_7fapfg
running install_egg_info


/home/kali/miniconda3/envs/bie/lib/python3.9/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpi_7fapfg/tfx_user_code_Trainer-0.0+06867b8317b6dcf37dabfe65babc104e2795971b5ca111d0cef24b5dadf85a97-py3.9.egg-info
running install_scripts
creating /tmp/tmpi_7fapfg/tfx_user_code_Trainer-0.0+06867b8317b6dcf37dabfe65babc104e2795971b5ca111d0cef24b5dadf85a97.dist-info/WHEEL
creating '/tmp/tmpnmgvts25/tfx_user_code_Trainer-0.0+06867b8317b6dcf37dabfe65babc104e2795971b5ca111d0cef24b5dadf85a97-py3-none-any.whl' and adding '/tmp/tmpi_7fapfg' to it
adding 'constants.py'
adding 'trainer.py'
adding

/home/kali/miniconda3/envs/bie/lib/python3.9/site-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['Blast Furnace Slag_xf', 'Cement_xf', 'Coarse Aggregate_xf', 'Fine Aggregate_xf', 'Fly Ash_xf', 'Superplasticizer_xf', 'Water_xf'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


500/500 [==============================] - 2s 3ms/step - loss: -4726.1953 - mean_squared_error: 1521.8013 - val_loss: -14524.2197 - val_mean_squared_error: 1363.7656


/home/kali/miniconda3/envs/bie/lib/python3.9/site-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['Blast Furnace Slag_xf', 'Coarse Aggregate_xf', 'Water_xf', 'Superplasticizer_xf', 'Cement_xf', 'Fly Ash_xf', 'Fine Aggregate_xf'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


ExecutionResult(
    component_id: Trainer
    execution_id: 10
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [37]:
evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
context.run(evaluator, enable_cache=False)

ExecutionResult(
    component_id: Evaluator
    execution_id: 11
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [38]:
context.run(model_resolver, enable_cache=False)

ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 12
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [39]:
evaluator.outputs

{'evaluation': OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False),
 'blessing': OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)}

In [40]:
context.show(evaluator.outputs['evaluation'])

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

In [41]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)

In [42]:
blessing_uri = evaluator.outputs['blessing'].get()[0].uri
!ls -l {blessing_uri}

total 0
-rw-r--r-- 1 kali kali 0 Jun  1 21:20 NOT_BLESSED


In [43]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))

metric_validations_per_slice {
  slice_key {
  }
  failures {
    metric_key {
      name: "mean_squared_error"
      model_name: "candidate"
      is_diff: true
      example_weighted {
      }
    }
    metric_threshold {
      change_threshold {
        absolute {
          value: -1e-10
        }
        direction: HIGHER_IS_BETTER
      }
    }
    metric_value {
      double_value {
        value: -1.842954588937573e-08
      }
    }
  }
}
metric_validations_per_slice {
  slice_key {
    single_slice_keys {
      column: "Age"
      int64_value: 3
    }
  }
  failures {
    metric_key {
      name: "mean_squared_error"
      model_name: "candidate"
      is_diff: true
      example_weighted {
      }
    }
    metric_threshold {
      change_threshold {
        absolute {
          value: -1e-10
        }
        direction: HIGHER_IS_BETTER
      }
    }
    metric_value {
      double_value {
        value: -1.2286511719139526e-07
      }
    }
  }
}
validation_details {
  slici

In [44]:
_serving_model_dir = "serving_model_dir"


In [45]:
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher, enable_cache=True)

ExecutionResult(
    component_id: Pusher
    execution_id: 13
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [46]:
pusher.outputs

{'pushed_model': OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)}

In [47]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

tfma.view.render_slicing_metrics(
    tfma_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

In [48]:
!mkdir -p saved_model

In [66]:
import time
import trainer

model = trainer.get_model()
ts = int(time.time())
filepath = "./saved_model/{}".format(ts)
model.save('saved_model/my_model')

Model: "model_41"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Cement_xf (InputLayer)      [(None, 1)]               0         
                                                                 
 concatenate_41 (Concatenat  (None, 1)                 0         
 e)                                                              
                                                                 
 flatten_41 (Flatten)        (None, 1)                 0         
                                                                 
 dense_82 (Dense)            (None, 128)               256       
                                                                 
 dense_83 (Dense)            (None, 1)                 129       
                                                                 
Total params: 385 (1.50 KB)
Trainable params: 385 (1.50 KB)
Non-trainable params: 0 (0.00 Byte)
____________________________

In [63]:
"""import trainer
def model_fn(features, mode, labels, params):
    logits = trainer.get_model(features)
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=logits)
    else:
        labels = features.pop(transformed_label_key)
        loss = tf.keras.losses.binary_crossentropy(labels, logits)
        if mode == tf.estimator.ModeKeys.EVAL:
            return tf.estimator.EstimatorSpec(mode=mode, loss=loss)
        elif mode == tf.estimator.ModeKeys.TRAIN:
            optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
            train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
            return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    return model_fn"""

In [64]:
def serving_input_reciever_fn():
    input_feature = tf.compat.v1.placeholder(
        dtype=tf.float32, shape=[None,1], name="input"
    )
    fn = tf.estimator.export.build_raw_serving_input_receiver_fn(
        features={"input_feature": input_feature}
    )
    return fn

In [ ]:
estimator = serving_input_receiver_fn = tf.estimator.build_raw_serving_input_receiver_fn(
    {'input': tf.placeholder(tf.float32, shape=[None, 784])})
estimator.export_saved_model(export_dir_base="saved_model/",
                             serving_input_receiver_fn=serving_input_receiver_fn)